# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,May 26 2022 11:37AM,238159,18,HH6105,Hush Hush,Released
1,May 26 2022 11:01AM,238157,10,8061517,Eywa Pharma Inc.,Executing
2,May 26 2022 11:01AM,238157,10,8061518,Eywa Pharma Inc.,Released
3,May 26 2022 11:01AM,238157,10,8061519,Eywa Pharma Inc.,Executing
4,May 26 2022 11:01AM,238157,10,8061520,Eywa Pharma Inc.,Executing
5,May 26 2022 11:01AM,238157,10,8061521,Eywa Pharma Inc.,Executing
6,May 26 2022 11:01AM,238157,10,8061522,Eywa Pharma Inc.,Released
7,May 26 2022 11:01AM,238157,10,8061524,Eywa Pharma Inc.,Released
8,May 26 2022 11:01AM,238157,10,8061525,Eywa Pharma Inc.,Released
9,May 26 2022 11:01AM,238157,10,8061527,Eywa Pharma Inc.,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
32,238155,Released,1
33,238156,Released,1
34,238157,Executing,7
35,238157,Released,12
36,238159,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
238154,NaN,1.0,NaN
238155,NaN,NaN,1.0
238156,NaN,NaN,1.0
238157,NaN,7.0,12.0
238159,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238071,0.0,0.0,1.0
238087,0.0,0.0,1.0
238090,0.0,18.0,11.0
238100,0.0,0.0,1.0
238101,0.0,0.0,20.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238071,0,0,1
238087,0,0,1
238090,0,18,11
238100,0,0,1
238101,0,0,20


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238071,0,0,1
1,238087,0,0,1
2,238090,0,18,11
3,238100,0,0,1
4,238101,0,0,20


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238071,,,1
1,238087,,,1
2,238090,,18,11
3,238100,,,1
4,238101,,,20


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,May 26 2022 11:37AM,238159,18,Hush Hush
1,May 26 2022 11:01AM,238157,10,Eywa Pharma Inc.
20,May 26 2022 10:56AM,238156,18,"ClearSpec, LLC"
21,May 26 2022 10:52AM,238155,18,"ClearSpec, LLC"
22,May 26 2022 10:44AM,238154,12,Doctors Scientific Organica
23,May 26 2022 10:41AM,238153,20,"ACI Healthcare USA, Inc."
24,May 26 2022 9:47AM,238150,10,Beach Products Inc
41,May 26 2022 9:32AM,238147,20,Alumier Labs Inc.
44,May 26 2022 9:25AM,238146,20,Alumier Labs Inc.
64,May 26 2022 9:15AM,238139,15,"Person & Covey, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,May 26 2022 11:37AM,238159,18,Hush Hush,,,1
1,May 26 2022 11:01AM,238157,10,Eywa Pharma Inc.,,7,12
2,May 26 2022 10:56AM,238156,18,"ClearSpec, LLC",,,1
3,May 26 2022 10:52AM,238155,18,"ClearSpec, LLC",,,1
4,May 26 2022 10:44AM,238154,12,Doctors Scientific Organica,,1,
5,May 26 2022 10:41AM,238153,20,"ACI Healthcare USA, Inc.",,,1
6,May 26 2022 9:47AM,238150,10,Beach Products Inc,,,17
7,May 26 2022 9:32AM,238147,20,Alumier Labs Inc.,,,3
8,May 26 2022 9:25AM,238146,20,Alumier Labs Inc.,,,20
9,May 26 2022 9:15AM,238139,15,"Person & Covey, Inc.",,,22


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 26 2022 11:37AM,238159,18,Hush Hush,1,,
1,May 26 2022 11:01AM,238157,10,Eywa Pharma Inc.,12,7,
2,May 26 2022 10:56AM,238156,18,"ClearSpec, LLC",1,,
3,May 26 2022 10:52AM,238155,18,"ClearSpec, LLC",1,,
4,May 26 2022 10:44AM,238154,12,Doctors Scientific Organica,,1,
5,May 26 2022 10:41AM,238153,20,"ACI Healthcare USA, Inc.",1,,
6,May 26 2022 9:47AM,238150,10,Beach Products Inc,17,,
7,May 26 2022 9:32AM,238147,20,Alumier Labs Inc.,3,,
8,May 26 2022 9:25AM,238146,20,Alumier Labs Inc.,20,,
9,May 26 2022 9:15AM,238139,15,"Person & Covey, Inc.",22,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 26 2022 11:37AM,238159,18,Hush Hush,1,,
1,May 26 2022 11:01AM,238157,10,Eywa Pharma Inc.,12,7,
2,May 26 2022 10:56AM,238156,18,"ClearSpec, LLC",1,,
3,May 26 2022 10:52AM,238155,18,"ClearSpec, LLC",1,,
4,May 26 2022 10:44AM,238154,12,Doctors Scientific Organica,,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 26 2022 11:37AM,238159,18,Hush Hush,1.0,NaN,NaN
1,May 26 2022 11:01AM,238157,10,Eywa Pharma Inc.,12.0,7.0,NaN
2,May 26 2022 10:56AM,238156,18,"ClearSpec, LLC",1.0,NaN,NaN
3,May 26 2022 10:52AM,238155,18,"ClearSpec, LLC",1.0,NaN,NaN
4,May 26 2022 10:44AM,238154,12,Doctors Scientific Organica,NaN,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 26 2022 11:37AM,238159,18,Hush Hush,1.0,0.0,0.0
1,May 26 2022 11:01AM,238157,10,Eywa Pharma Inc.,12.0,7.0,0.0
2,May 26 2022 10:56AM,238156,18,"ClearSpec, LLC",1.0,0.0,0.0
3,May 26 2022 10:52AM,238155,18,"ClearSpec, LLC",1.0,0.0,0.0
4,May 26 2022 10:44AM,238154,12,Doctors Scientific Organica,0.0,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1190637,36.0,28.0,7.0
12,238154,0.0,1.0,0.0
15,1190698,35.0,6.0,0.0
16,714351,2.0,1.0,0.0
18,952603,6.0,8.0,0.0
19,714323,5.0,15.0,21.0
20,1190637,55.0,18.0,0.0
21,238121,1.0,0.0,0.0
22,476226,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1190637,36.0,28.0,7.0
1,12,238154,0.0,1.0,0.0
2,15,1190698,35.0,6.0,0.0
3,16,714351,2.0,1.0,0.0
4,18,952603,6.0,8.0,0.0
5,19,714323,5.0,15.0,21.0
6,20,1190637,55.0,18.0,0.0
7,21,238121,1.0,0.0,0.0
8,22,476226,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,36.0,28.0,7.0
1,12,0.0,1.0,0.0
2,15,35.0,6.0,0.0
3,16,2.0,1.0,0.0
4,18,6.0,8.0,0.0
5,19,5.0,15.0,21.0
6,20,55.0,18.0,0.0
7,21,1.0,0.0,0.0
8,22,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,36.0
1,12,Released,0.0
2,15,Released,35.0
3,16,Released,2.0
4,18,Released,6.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,18,19,20,21,22
Status,,,,,,,,,
Completed,7.0,0.0,0.0,0.0,0.0,21.0,0.0,0.0,0.0
Executing,28.0,1.0,6.0,1.0,8.0,15.0,18.0,0.0,0.0
Released,36.0,0.0,35.0,2.0,6.0,5.0,55.0,1.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,18,19,20,21,22
0,Completed,7.0,0.0,0.0,0.0,0.0,21.0,0.0,0.0,0.0
1,Executing,28.0,1.0,6.0,1.0,8.0,15.0,18.0,0.0,0.0
2,Released,36.0,0.0,35.0,2.0,6.0,5.0,55.0,1.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,18,19,20,21,22
0,Completed,7.0,0.0,0.0,0.0,0.0,21.0,0.0,0.0,0.0
1,Executing,28.0,1.0,6.0,1.0,8.0,15.0,18.0,0.0,0.0
2,Released,36.0,0.0,35.0,2.0,6.0,5.0,55.0,1.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()